In [ ]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
import copy

In [ ]:
torch.manual_seed(777)
torch.cuda.manual_seed_all(777)

GPU_NUM = 1 # 사용 할 GPU Num 설정
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)

print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())
print(torch.cuda.get_device_name(device))

print("cpu와 cuda 중 다음 기기로 학습함:", device, '\n')

In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square conv kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # 5x5 image dimension
        self.fc2 = nn.Linear(120, 20)
        self.fc3 = nn.Linear(20, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, int(x.nelement() / x.shape[0]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
# 서로 다른 2개의 model 생성
model1 = LeNet().to(device)
model2 = LeNet().to(device)

In [5]:
model1_1 = copy.deepcopy(model1)

In [6]:
model1.fc3.bias

Parameter containing:
tensor([ 1.3251e-01, -4.7817e-02,  8.1092e-02, -1.8831e-01,  5.8762e-02,
         7.4819e-05,  9.1043e-02,  8.8944e-02, -2.0730e-01,  1.9853e-01],
       device='cuda:1', requires_grad=True)

In [7]:
prune.l1_unstructured(model1.fc3, name='bias', amount=0.2)

Linear(in_features=20, out_features=10, bias=True)

In [8]:
model1.fc3.bias

tensor([ 0.1325, -0.0000,  0.0811, -0.1883,  0.0588,  0.0000,  0.0910,  0.0889,
        -0.2073,  0.1985], device='cuda:1', grad_fn=<MulBackward0>)

In [9]:
model2.fc3.bias

Parameter containing:
tensor([ 0.0701,  0.1277, -0.2231,  0.1966, -0.0440, -0.1797, -0.1639, -0.1986,
        -0.0497, -0.0235], device='cuda:1', requires_grad=True)

In [10]:
model1.fc3.bias_orig

Parameter containing:
tensor([ 1.3251e-01, -4.7817e-02,  8.1092e-02, -1.8831e-01,  5.8762e-02,
         7.4819e-05,  9.1043e-02,  8.8944e-02, -2.0730e-01,  1.9853e-01],
       device='cuda:1', requires_grad=True)

In [12]:
model1.fc3.bias_mask

tensor([1., 0., 1., 1., 1., 0., 1., 1., 1., 1.], device='cuda:1')

In [18]:
model1.fc3.bias_orig = copy.deepcopy(model2.fc3.bias)

In [14]:
model1.fc3.bias_orig

Parameter containing:
tensor([ 0.0701,  0.1277, -0.2231,  0.1966, -0.0440, -0.1797, -0.1639, -0.1986,
        -0.0497, -0.0235], device='cuda:1', requires_grad=True)

In [15]:
prune.remove(model1.fc3, "bias")

Linear(in_features=20, out_features=10, bias=True)

In [17]:
model1.fc3.bias

Parameter containing:
tensor([ 0.0701,  0.0000, -0.2231,  0.1966, -0.0440, -0.0000, -0.1639, -0.1986,
        -0.0497, -0.0235], device='cuda:1', requires_grad=True)

In [ ]:
print(model1.state_dict().keys())

In [ ]:
print(model1.state_dict().keys())

In [ ]:
model1.fc3.bias

In [ ]:
model1.fc3.bias_mask

In [ ]:
model1.fc3._forward_pre_hooks

In [ ]:
model1.fc3.bias

In [ ]:
prune.remove(model1.fc3, "bias")

In [ ]:
model1.fc3.bias

In [ ]:
print(model1.state_dict().keys())

In [ ]:
model1.state_dict()

In [ ]:
model1.conv1.weight_orig = model5.conv1.weight

In [ ]:
model1.state_dict()

In [ ]:
print(module1._forward_pre_hooks)


In [ ]:
module1.weight

In [ ]:
# model1의 state를 copy
model3 = copy.deepcopy(model1.state_dict())

In [ ]:
model3

In [ ]:
module1 = model1.conv1
prune.random_unstructured(module1, name='weight', amount=0.3)
prune.random_unstructured(module1, name='bias', amount=0.3)

In [ ]:
print(model1.state_dict().keys())

In [ ]:
model1.load_state_dict(model3.state_dict())

In [ ]:
model2

In [ ]:
module1 = model1.conv1
print(list(module1.weight))

In [ ]:
module2 = model2.conv1
print(list(module2.named_parameters()))

In [ ]:
# model1 conv1에 대하여 random pruning 진행
prune.random_unstructured(module1, name='weight', amount=0.3)
prune.random_unstructured(module1, name='bias', amount=0.3)

In [ ]:
# mask 확인
print(model1.state_dict().keys())

In [ ]:
# mask에 따라 pruning된 parameter 확인
print(module1.weight)

In [ ]:
print(module1.bias)

In [ ]:
print(model1.state_dict().keys())

In [ ]:
print(module1.state_dict().keys())

In [ ]:
print(model2.conv1.bias_mask)

In [ ]:
class FooBarPruningMethod(prune.BasePruningMethod):
    """Prune every other entry in a tensor
    """
    PRUNING_TYPE = 'structured'

    def compute_mask(self, t, default_mask):
        mask = default_mask.clone()
        mask.view(-1)[::2] = 0
        return mask
    
def foobar_unstructured(module, name):
    """Prunes tensor corresponding to parameter called `name` in `module`
    by removing every other entry in the tensors.
    Modifies module in place (and also return the modified module)
    by:
    1) adding a named buffer called `name+'_mask'` corresponding to the
    binary mask applied to the parameter `name` by the pruning method.
    The parameter `name` is replaced by its pruned version, while the
    original (unpruned) parameter is stored in a new parameter named
    `name+'_orig'`.

    Args:
        module (nn.Module): module containing the tensor to prune
        name (string): parameter name within `module` on which pruning
                will act.

    Returns:
        module (nn.Module): modified (i.e. pruned) version of the input
            module

    Examples:
        >>> m = nn.Linear(3, 4)
        >>> foobar_unstructured(m, name='bias')
    """
    FooBarPruningMethod.apply(module, name)
    return module

In [ ]:
foobar_unstructured(model2.conv1, name='bias')
print(model2.conv1.bias_mask)

In [ ]:
model2.conv1.bias_mask = model1.conv1.bias_mask

In [ ]:
print(model2.conv1.bias_mask)

In [ ]:
print(module1.state_dict().keys())

In [ ]:
print(module2.state_dict().keys())

In [ ]:
print(module2.bias_mask)

In [ ]:
module2.bias = module1.bias

In [ ]:
print(module2.bias)

In [ ]:
print(module2.bias)

In [ ]:
print(module2._forward_pre_hooks)

In [ ]:
print(module2.bias)

In [ ]:
foobar_unstructured(model2.conv1, name='weight')

In [ ]:
print(model2.conv1.weight_mask)

In [ ]:
print(module2.state_dict().keys())

In [ ]:
model2.conv1.weight_mask = model1.conv1.weight_mask

In [ ]:
print(model2.conv1.weight_mask)

In [ ]:
print(model2.conv1.weight)

In [ ]:
print(module1.weight)

In [ ]:
print()